In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import importlib
import scraper_utils as su
%load_ext autoreload
%autoreload 2
import time

from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data in

Load data in from csvs, convert categories and types columns to lists (saved as strings)

In [37]:
cols = ['Name', 'id', 'baverage', 'average','numwish', 'numowned', 'numplays','numplays_month','numfans', 'avgweight',  'yearpublished',
       'minplayers', 'maxplayers', 'minplaytime', 'maxplaytime', 'minage', 'categories', 'types']
df_list = []
for i in range(10):
    fname=f'data/partial_data/df{i}.csv'
    df = pd.read_csv(fname,usecols = cols)
    df_list.append(df)

df = pd.concat(df_list, ignore_index=True)
df['categories']= df['categories'].apply(literal_eval)
df['types']= df['types'].apply(literal_eval)


In [10]:
with open('col_names.pkl', 'wb') as pf:
    pickle.dump(cols, pf)

## Convert categories from code to names

In [38]:
category_dict = su.make_category_dict()

In [39]:
category_dict = su.drop_categories(category_dict, ['World War II','World War I','Modern Warfare','American Civil War',
                                                  'American Revolutionary War', 'Space Exploration','Vietnam War'])

In [40]:
with open('category_dict.pkl', 'wb') as pf:
    pickle.dump(category_dict, pf)

In [41]:
df['categories']=df['categories'].apply(lambda x: su.code_to_name(category_dict,x))

In [42]:
df

,Name,id,average,baverage,avgweight,numwish,numowned,numplays,numplays_month,numfans,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,categories,types
0,Gloomhaven,174430,8.79923,8.55546,3.8549,15900.0,66360.0,332850.0,3066.0,7003.0,2017.0,1.0,4.0,60.0,120.0,14.0,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[thematic, strategy]"
1,Pandemic Legacy: Season 1,161936,8.61526,8.46581,2.8400,10696.0,64193.0,228424.0,1134.0,2779.0,2015.0,2.0,4.0,60.0,60.0,13.0,"[Environmental, Medical]","[thematic, strategy]"
2,Brass: Birmingham,224517,8.66068,8.36360,3.9170,9226.0,27062.0,43737.0,801.0,1459.0,2018.0,2.0,4.0,60.0,120.0,14.0,"[Economic, Industry / Manufacturing, Transport...",[strategy]
3,Terraforming Mars,167791,8.43238,8.28551,3.2388,17193.0,84252.0,349382.0,4080.0,5550.0,2016.0,1.0,5.0,120.0,120.0,12.0,"[Economic, Environmental, Industry / Manufactu...",[strategy]
4,Twilight Imperium: Fourth Edition,233078,8.69556,8.24081,4.2224,7813.0,16328.0,18830.0,232.0,1582.0,2017.0,3.0,6.0,240.0,480.0,14.0,"[Civilization, Economic, Negotiation, Politica...","[thematic, strategy]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Let's Take a Hike,104063,5.89506,5.54878,1.3500,69.0,328.0,473.0,12.0,12.0,2011.0,2.0,5.0,20.0,30.0,5.0,"[Card Game, Children's Game]",[family]
9996,Doodle China,205790,6.47051,5.54876,2.0000,47.0,129.0,242.0,3.0,30.0,2016.0,1.0,5.0,20.0,40.0,8.0,[City Building],[]
9997,Web and Starship,2986,6.31991,5.54876,3.5333,86.0,352.0,48.0,1.0,12.0,1984.0,2.0,3.0,120.0,120.0,12.0,"[Science Fiction, Wargame]",[war]
9998,Slyville,266936,6.85562,5.54876,1.5000,54.0,106.0,129.0,4.0,41.0,2019.0,3.0,5.0,45.0,60.0,10.0,"[Bluffing, Humor, Medieval, Negotiation]",[]


## Get marketplace data

Iterate through all ids, get marketplace data, keep track of which ids didn't have any for sale (in USD)

In [18]:
all_ids = df['id']

In [52]:
marketplace_list = []
no_listings = []
start = time.time()   
for i,id_ in enumerate(all_ids):
        mp_data = su.get_marketplace_data(id_)
        if mp_data[0]==0: no_listings.append(id_)
        marketplace_list.append(mp_data)
        if i%500 == 0:
            print(i, 'done, last 500 took', str(time.time()-start))
            start = time.time()

0 done, last 500 took 0.1080179214477539
500 done, last 500 took 34.67627716064453
1000 done, last 500 took 31.44352102279663
1500 done, last 500 took 30.87442684173584
2000 done, last 500 took 30.23093605041504
2500 done, last 500 took 29.912564992904663
3000 done, last 500 took 29.44926118850708
3500 done, last 500 took 32.20809197425842
4000 done, last 500 took 28.949753999710083
4500 done, last 500 took 29.182222843170166
5000 done, last 500 took 29.20558500289917
5500 done, last 500 took 68.37382674217224
6000 done, last 500 took 76.51822209358215
6500 done, last 500 took 76.23167276382446
7000 done, last 500 took 78.71208596229553
7500 done, last 500 took 79.2893226146698
8000 done, last 500 took 74.49512815475464
8500 done, last 500 took 74.36403894424438
9000 done, last 500 took 71.50854897499084
9500 done, last 500 took 69.66451907157898


Save as a DataFrame and convert to csv

In [118]:
mp_df = pd.DataFrame(marketplace_list, columns = ['num_listings','median_price'])
mp_df.to_csv('data/partial_data/marketplace_data.csv', index=False)


In [43]:
mp_df=pd.read_csv('data/partial_data/marketplace_data.csv')

## Combine marketplace data and other stats into 1 df

Also do cleaning - get rid of things with low owned and num plays values, and with no marketplace data. Also filter by games that have come out since 2010 - this is what actually does more of the filtering. This leaves us with less than half the data we started with. That's ok because we still have almost 5000 entries and most of what we got rid of is old.

In [44]:
df_with_mp = pd.concat([df,mp_df], axis=1)

In [45]:
df_with_mp.sort_values('minage',ascending=False).head(20)

,Name,id,average,baverage,avgweight,numwish,numowned,numplays,numplays_month,numfans,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,categories,types,num_listings,median_price
6856,Das Wasser des Lebens,5549,7.62765,5.60545,2.6667,65.0,159.0,21.0,6.0,60.0,1997.0,3.0,4.0,60.0,60.0,21.0,[Civilization],[],1,198.99
9561,1836,273636,7.82432,5.55396,0.0000,52.0,77.0,81.0,0.0,1.0,2019.0,3.0,5.0,90.0,150.0,21.0,"[Economic, Trains, Transportation]",[],0,0.00
6493,DrunkQuest,125493,6.27465,5.61623,1.9412,60.0,1157.0,352.0,1.0,49.0,2013.0,3.0,6.0,20.0,20.0,21.0,"[Card Game, Fantasy, Humor, Party Game]",[party],0,0.00
9950,Unstable Unicorns: NSFW Base Game,287952,7.03789,5.54932,2.0000,36.0,407.0,168.0,8.0,50.0,2019.0,2.0,8.0,30.0,45.0,21.0,"[Card Game, Humor, Mature / Adult, Party Game]",[],0,0.00
9750,Drinking Fluxx,211996,6.27370,5.55185,1.0000,92.0,679.0,254.0,7.0,71.0,2017.0,2.0,6.0,10.0,40.0,21.0,[Card Game],[],2,12.50
6060,Swinging Jivecat Voodoo Lounge,99655,6.56472,5.63264,2.6250,105.0,613.0,533.0,1.0,170.0,2016.0,2.0,5.0,60.0,60.0,21.0,"[Mature / Adult, Number, Territory Building]",[],2,42.50
7157,Chez Greek,4604,5.97200,5.59765,1.4595,34.0,974.0,247.0,1.0,80.0,2002.0,2.0,6.0,60.0,60.0,18.0,"[Card Game, Humor]",[thematic],1,49.00
5970,Ferox,183836,7.82238,5.63631,2.1429,103.0,218.0,286.0,39.0,39.0,2015.0,2.0,2.0,60.0,90.0,18.0,"[Adventure, Card Game, Fighting, Horror]",[],2,107.50
6038,Consentacle,166976,6.83676,5.63349,1.3333,120.0,780.0,279.0,37.0,37.0,2014.0,2.0,2.0,10.0,35.0,18.0,"[Card Game, Mature / Adult, Science Fiction]",[],0,0.00
6037,Monikers,283150,8.00679,5.63353,1.0000,60.0,251.0,204.0,6.0,20.0,2019.0,4.0,20.0,60.0,60.0,18.0,[Party Game],[],0,0.00


In [46]:
cleaning_mask = (df_with_mp['numowned'] > 0)&\
                (df_with_mp['numplays'] > 0)&\
                (df_with_mp['minplayers'] > 0)&\
                (df_with_mp['maxplayers'] <= 10)&\
                (df_with_mp['minplaytime'] > 0)&\
                (df_with_mp['maxplaytime'] <= 600)&\
                (df_with_mp['types'].apply(lambda x: len(x)) > 0)
df_cleaned=df_with_mp[cleaning_mask].copy()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6400 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            6400 non-null   object 
 1   id              6400 non-null   int64  
 2   average         6400 non-null   float64
 3   baverage        6400 non-null   float64
 4   avgweight       6400 non-null   float64
 5   numwish         6400 non-null   float64
 6   numowned        6400 non-null   float64
 7   numplays        6400 non-null   float64
 8   numplays_month  6400 non-null   float64
 9   numfans         6400 non-null   float64
 10  yearpublished   6400 non-null   float64
 11  minplayers      6400 non-null   float64
 12  maxplayers      6400 non-null   float64
 13  minplaytime     6400 non-null   float64
 14  maxplaytime     6400 non-null   float64
 15  minage          6400 non-null   float64
 16  categories      6400 non-null   object 
 17  types           6400 non-null   o

In [25]:
#def ohe_categories(df, cols_to_drop):
#    """
#    One-hot encodes game categories, then drops any columns in cols_to drop
#    
#    """
#    mlb = MultiLabelBinarizer()
#    df2 = df.join(pd.DataFrame(mlb.fit_transform(df.pop('categories')),
#                          columns=mlb.classes_,
#                          index=df.index))
#    for col in cols_to_drop:
#        if col in df2.columns:
#            df2.drop(columns=[col], inplace=True)
#    return df2

def ohe_categories(df, cols_to_keep, cols_to_drop):
    """
    One-hot encodes game categories, then drops any columns in cols_to drop
    
    """
    mlb = MultiLabelBinarizer()
    ohe_df=pd.DataFrame(mlb.fit_transform(df['categories']),
                          columns=mlb.classes_,
                          index=df.index)
    #Makes sure all columns in cols to keep are in the df, even if they're zeroz
    for col in cols_to_keep:
        if col not in ohe_df.columns:
            ohe_df[col] = 0
    ohe_df = ohe_df.reindex(columns=sorted(ohe_df.columns))
    
    #Drops cols in cols_to_drop, usually used for the least popular tags
    for col in cols_to_drop:
        if col in ohe_df.columns:
            ohe_df.drop(columns=[col], inplace=True)
    return df.join(ohe_df).drop(columns=['categories'])

def ohe_types(df):
    """
    One-hot encodes game types, then drops the types that repeat with categories
    """
    mlb = MultiLabelBinarizer()
    df2 = df.join(pd.DataFrame(mlb.fit_transform(df.pop('types')),
                          columns=mlb.classes_,
                          index=df.index))
    
    #Makes sure the columns with useful information are always there
    for col in ['customizable', 'family', 'strategy', 'thematic']:
        if col not in df2.columns:
            df2[col] = 0
    
    #Drops columns with not useful info
    for col in ['abstract', 'children\'s', 'party','war' ]:
        if col in df2.columns:
            df2.drop(columns=[col], inplace=True)
    return df2

In [47]:
category_list = list(category_dict.values())
df_type_ohe = ohe_types(df_cleaned)
df_top10 = ohe_categories(df_type_ohe, category_list[:10], category_list[10:])
df_top20 = ohe_categories(df_type_ohe, category_list[:20], category_list[20:])
df_top30 = ohe_categories(df_type_ohe, category_list[:30], [])
df_all   = ohe_categories(df_type_ohe, category_list,[])

In [48]:
df_top10.to_csv('data/games_top10.csv', index=False)
df_top20.to_csv('data/games_top20.csv', index=False)
df_top30.to_csv('data/games_top30.csv', index=False)
df_all.to_csv('data/games_allcats.csv', index=False)

In [49]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6400 entries, 0 to 9999
Data columns (total 99 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Name                       6400 non-null   object 
 1   id                         6400 non-null   int64  
 2   average                    6400 non-null   float64
 3   baverage                   6400 non-null   float64
 4   avgweight                  6400 non-null   float64
 5   numwish                    6400 non-null   float64
 6   numowned                   6400 non-null   float64
 7   numplays                   6400 non-null   float64
 8   numplays_month             6400 non-null   float64
 9   numfans                    6400 non-null   float64
 10  yearpublished              6400 non-null   float64
 11  minplayers                 6400 non-null   float64
 12  maxplayers                 6400 non-null   float64
 13  minplaytime                6400 non-null   float

In [24]:
len(category_list)

NameError: name 'category_list' is not defined